In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%time

import numpy as np
import pandas as pd
import time
import pickle
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from sklearn.model_selection import KFold


CPU times: total: 859 ms
Wall time: 2.46 s


In [3]:
%%time
start_time = time.time()
path = "C:/Users/nazih/Downloads/ember/"
train_size = 800000
test_size = 200000
columns = 2381
X_train = np.memmap(path+"X_train.dat", dtype=np.float32, mode="r", shape=(train_size, columns))
y_train = np.memmap(path+"y_train.dat", dtype=np.float32, mode="r", shape=train_size)
X_test = np.memmap(path+"X_test.dat", dtype=np.float32, mode="r", shape=(test_size, columns))
y_test = np.memmap(path+"y_test.dat", dtype=np.float32, mode="r", shape=test_size)

train_rows = (y_train != -1)
X_train=X = X_train[train_rows]
y_train=y = y_train[train_rows]
print("Processing time: %s seconds " % (time.time() - start_time))


Processing time: 43.72270178794861 seconds 
CPU times: total: 8.55 s
Wall time: 43.8 s


In [5]:
#Redure dataset to run on Kaggle if needed. 
for i in range(2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    X, y  = X_train, y_train
    print(X.shape, y.shape)

(402000, 2381) (402000,)
(269340, 2381) (269340,)


In [6]:
%%time
X = pd.DataFrame(X_train)
y = pd.Series(y_train)
X = X.astype(np.float32)
y = y.astype(np.float32)

CPU times: total: 24.3 s
Wall time: 1min 56s


In [ ]:
#print(X.shape, y.shape)
X.head(5)

,0,1,2,3,4,5,6,7,8,9,...,2371,2372,2373,2374,2375,2376,2377,2378,2379,2380
0,0.167156,0.012020,0.007193,0.008241,0.010548,0.004110,0.003892,0.003796,0.012897,0.002395,...,64.0,189056.0,0.0,0.0,552.0,176128.0,288.0,193012.0,0.0,0.0
1,0.289915,0.011604,0.009505,0.012839,0.009782,0.003218,0.001223,0.001130,0.006287,0.001949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.582955,0.012451,0.003372,0.003378,0.006737,0.001782,0.001755,0.001511,0.006717,0.001043,...,64.0,67512.0,0.0,0.0,316.0,61440.0,0.0,0.0,0.0,0.0
3,0.604688,0.014453,0.005078,0.002734,0.009375,0.001172,0.001563,0.003125,0.003516,0.000781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.248930,0.024164,0.013127,0.013370,0.012738,0.009675,0.018329,0.005591,0.006709,0.005299,...,0.0,0.0,0.0,0.0,8.0,8192.0,0.0,0.0,72.0,8200.0


In [7]:
#release memory
del X_train
del y_train

In [8]:
print(X.info(), X.shape, y.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269340 entries, 0 to 269339
Columns: 2381 entries, 0 to 2380
dtypes: float32(2381)
memory usage: 2.4 GB
None (269340, 2381) (269340,)


![](http://)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


# Define your classifiers
log_reg = LogisticRegression(penalty='l2',C=0.001,solver='saga',random_state=1, max_iter=1000)
forest = RandomForestClassifier(random_state=1,n_estimators=200,max_depth=10,class_weight='balanced')
bayes = GaussianNB()
ensemble = VotingClassifier(estimators=[
    ('lr', log_reg),
    ('rf', forest),
    ('gnb', bayes)
], voting='soft')

# List of classifiers and their labels
classifiers = [log_reg, forest, bayes, ensemble]
labels = ['Logistic Regression', 'Random Forest', 'Gaussian Naive Bayes', 'Ensemble']

# Perform cross-validation and print accuracy
for clf, label in zip(classifiers, labels):
    scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc_ovr')
    print(f"Accuracy: {scores.mean():.2f} (+/- {scores.std():.2f}) [{label}]")

# Fit the ensemble model
ensemble.fit(X, y)

# Predict probabilities and evaluate
y_pred_probs_ensemble = ensemble.predict_proba(X_test)
roc_auc_ensemble = roc_auc_score(y_test, y_pred_probs_ensemble, multi_class="ovr")
 
print(f"ROC AUC for Ensemble (cv dataset): {roc_auc_ensemble:.2f}")